In [1]:
#Import relevant libraries
import astropy
import numpy as np
import ccdproc
import os
import glob
import matplotlib.pyplot as plt
from astropy.nddata import CCDData
import scipy

In [2]:
#TASK 1
#We do the same as previously stated in Science Frames B1 and B2 
#Subtract master bias and master dark from the first half of our science V

masterflatV = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expflatV.fits', unit='adu')
masterbias = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expbias.fits', unit='adu')
masterdark = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expdark.fits', unit='adu')


#file_location = os.path.join('20191111/', 'science_B/', '*.fits')
#file_location = os.path.join('GlobularClusters/', 'science_B/', '*.fits')
filenames = glob.glob('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/20191111/science_V/*.fits')

scienceimagesV = []

for file in filenames:
    scienceImg = astropy.nddata.CCDData.read(file, unit='adu')
    hdul = astropy.nddata.CCDData.to_hdu(scienceImg)
    scienceSub1 = scienceImg.subtract(masterbias)
    scienceSub2 = scienceSub1.subtract(masterdark)
    divscience = scienceSub2.divide(masterflatV)
    #divscience2 = divscience.divide(60) #to ensure exposure time is 1 sec
    scienceimagesV.append(divscience)
    
    
print(scienceimagesV)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


[CCDData([[ 313.09110013,  350.3057212 ,  324.93828943, ...,  304.55851812,
           333.0984145 ,  -42.12317569],
         [ 347.87068938,  382.44247015,  342.67949904, ...,  359.45865205,
           339.8826561 , -164.82233313],
         [ 335.829633  ,  306.7821142 ,  318.60899488, ...,  360.89046771,
           372.35326794,  186.77701767],
         ...,
         [ 318.8469171 ,  290.46262283,  308.27943009, ...,  299.04523327,
           296.53372195, -104.59788694],
         [ 306.07782196,  320.63173686,  329.285265  , ...,  362.20875091,
           325.39811387,  219.97118544],
         [ 337.78252854,  310.28162328,  370.65855706, ...,  338.94152101,
           346.05937234,  -50.90991734]], unit=''), CCDData([[ 346.93110097,  306.38381518,  314.88050466, ...,  354.70952871,
           302.59309442,  -80.58346654],
         [ 321.49599016,  350.13963707,  339.65394955, ...,  321.40253207,
           353.53927779, -190.46136272],
         [ 333.80699808,  326.91783666,  339.6

In [3]:
from skimage import data, draw
from skimage.registration import phase_cross_correlation
from scipy import ndimage

#We must align all the individual science frames in the V band 

scienceimagesVcorr = []
for i in range(len(scienceimagesV)):
    detected_shiftV = phase_cross_correlation(np.array(scienceimagesV[0]), np.array(scienceimagesV[i]), upsample_factor = 1000)
    print(detected_shiftV[0])
    scienceimagesV1 = scipy.ndimage.shift(scienceimagesV[i], detected_shiftV[0])
    scienceimagesV11 = astropy.nddata.CCDData(scienceimagesV1, unit = 'adu')
    scienceimagesVcorr.append(scienceimagesV11)

[0. 0.]
[-0.502  2.599]
[-0.332  2.523]
[-0.929  3.666]
[-1.241  5.583]
[-1.537  4.985]
[-1.9    8.538]
[-1.948  8.32 ]
[-2.029  9.493]
[-2.487 12.133]
[-1.65  10.087]
[-2.257  9.07 ]
[-1.889 10.432]
[-1.966  9.153]
[-2.31  11.264]


In [4]:
scienceimagesVcorr = astropy.nddata.CCDData(scienceimagesVcorr, unit='adu')
print(scienceimagesVcorr)

[[[ 313.09110013  350.3057212   324.93828943 ...  304.55851812
    333.0984145   -42.12317569]
  [ 347.87068938  382.44247015  342.67949904 ...  359.45865205
    339.8826561  -164.82233313]
  [ 335.829633    306.7821142   318.60899488 ...  360.89046771
    372.35326794  186.77701767]
  ...
  [ 318.8469171   290.46262283  308.27943009 ...  299.04523327
    296.53372195 -104.59788694]
  [ 306.07782196  320.63173686  329.285265   ...  362.20875091
    325.39811387  219.97118544]
  [ 337.78252854  310.28162328  370.65855706 ...  338.94152101
    346.05937234  -50.90991734]]

 [[   0.            0.            0.         ...  338.63144518
    334.03405097  303.90772792]
  [   0.            0.            0.         ...  342.38787864
    335.03343336  320.12780199]
  [   0.            0.            0.         ...  339.71437839
    334.44328886  347.2155819 ]
  ...
  [   0.            0.            0.         ...  310.33957194
    333.91560525  311.93291669]
  [   0.            0.            0.

In [5]:
print((scienceimagesVcorr.shape))

(15, 4096, 4096)


In [6]:
#Here we combine all alligned master science frames for the first half after having the master bias and dark subtracted and the masterflat divided
ScienceV = ccdproc.combine((scienceimagesVcorr), method = 'median', format = 'fits')
hdu1 = CCDData.to_hdu(ScienceV)
hdu1.writeto('MasterScienceV1.fits', overwrite = True)

INFO: splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]
